<a href="https://colab.research.google.com/github/nitinsb/Intern_media/blob/main/ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from transformers import ViTFeatureExtractor, ViTModel
import torch
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, Flatten
from tensorflow.keras.optimizers import Adam



In [ ]:
import pandas as pd
import os
folder_path = '/content/drive/MyDrive/Folder 1/Interview December 2023'
if os.path.isdir(folder_path):
  print(f"The path '{folder_path}' exists.")
else:
  print(f"The path '{folder_path}' does not exist.")



file_path_info = '/content/drive/MyDrive/Folder 1/Interview December 2023/instagram_data.csv'

if os.path.isfile(file_path_info):
  try:
    df = pd.read_csv(file_path_info)
    likes_stats = df['likes'].describe()
    print(likes_stats)
  except Exception as e:
    print(f"An error occurred: {e}")
else:
  print(f"The file '{file_path_info}' does not exist.")

The path '/content/drive/MyDrive/Folder 1/Interview December 2023' exists.
count    3.785000e+03
mean     1.832536e+05
std      1.936969e+05
min      1.431000e+03
25%      5.208700e+04
50%      1.236640e+05
75%      2.431440e+05
max      2.161369e+06
Name: likes, dtype: float64


In [ ]:
Q1 = df['likes'].quantile(0.05)
Q3 = df['likes'].quantile(0.95)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers
df_no_outliers = df[(df['likes'] >= lower_bound) & (df['likes'] <= upper_bound)]

# Print the shape of the original and outlier-removed dataframes
print(f"Original DataFrame shape: {df.shape}")
print(f"DataFrame shape after removing outliers: {df_no_outliers.shape}")
X = df_no_outliers[['likes','no_of_comments', 't', 'follower_count_at_t','image_path']]
X.shape



Original DataFrame shape: (3785, 5)
DataFrame shape after removing outliers: (3779, 5)


(3779, 5)

In [ ]:
threshold_low = X['likes'].quantile(0.33)
threshold_high = X['likes'].quantile(0.66)

# Create the new column 'y' based on the thresholds
def classify_likes(likes):
  if likes <= threshold_low:
    return 0
  elif likes <= threshold_high:
    return 1
  else:
    return 2

X['y'] = X['likes'].apply(classify_likes)

print(X['y'].value_counts())

y
2    1285
1    1247
0    1247
Name: count, dtype: int64


In [ ]:
X_y = X.drop('likes', axis=1)
X_y.head(50)


,no_of_comments,t,follower_count_at_t,image_path,y
0,0,1594174009,40934474,../Data/insta_data/0.jpg,1
1,0,1593571666,40934474,../Data/insta_data/2.jpg,1
2,0,1593136341,40934474,../Data/insta_data/4.jpg,1
3,0,1592981047,40934474,../Data/insta_data/6.jpg,1
4,0,1592703461,40934474,../Data/insta_data/8.jpg,1
5,0,1592276125,40934474,../Data/insta_data/9.jpg,1
6,0,1591582341,40934474,../Data/insta_data/11.jpg,1
7,0,1589702043,40934474,../Data/insta_data/19.jpg,2
8,0,1588470412,40934474,../Data/insta_data/20.jpg,2
9,0,1587696339,40934474,../Data/insta_data/21.jpg,2


In [ ]:
%cd /content/drive/MyDrive/Folder 1/Interview December 2023/

model_vit = ViTModel.from_pretrained('google/vit-base-patch16-224')
feature_extractor_vit = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')

def get_vit_embedding(image_path):
  try:
    img = image.load_img(image_path, target_size=(224, 224)) # Resize for ViT
    print(f"Processing image {image_path}")
    inputs = feature_extractor_vit(images=img, return_tensors="pt")
    with torch.no_grad():
      outputs = model_vit(**inputs).last_hidden_state[:, 0, :] # Extract [CLS] token
    return outputs.numpy().flatten()
  except Exception as e:
    print(f"Error processing image {image_path}: {e}")
    return None

vit_embeddings = []
for image_path in X_y['image_path']:
  print(image_path)
  embedding = get_vit_embedding(image_path)
  if embedding is not None:
    vit_embeddings.append(embedding)
  else:
    vit_embeddings.append([0] * 768) # Or handle missing embeddings as needed


/content/drive/MyDrive/Folder 1/Interview December 2023


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
Processing image ../Data/insta_data/2417.jpg
../Data/insta_data/2425.jpg
Processing image ../Data/insta_data/2425.jpg
../Data/insta_data/2436.jpg
Processing image ../Data/insta_data/2436.jpg
../Data/insta_data/2437.jpg
Processing image ../Data/insta_data/2437.jpg
../Data/insta_data/2448.jpg
Processing image ../Data/insta_data/2448.jpg
../Data/insta_data/2451.jpg
Processing image ../Data/insta_data/2451.jpg
../Data/insta_data/2455.jpg
Processing image ../Data/insta_data/2455.jpg
../Data/insta_data/2456.jpg
Processing image ../Data/insta_data/2456.jpg
../Data/insta_data/2459.jpg
Processing image ../Data/insta_data/2459.jpg
../Data/insta_data/2460.jpg
Processing image ../Data/insta_data/2460.jpg
../Data/insta_data/2462.jpg
Processing image ../Data/insta_data/2462.jpg
../Data/insta_data/2465.jpg
Processing image ../Data/insta_data/2465.jpg
../Data/insta_data/2466.jpg
Processing image ../Data/insta_data/2466.jpg
../Data/insta_data/2467.jpg


In [ ]:


# Add the embeddings to the dataframe
X_y['vit_embedding'] = vit_embeddings

print(X_y.head())

X_y_emb_vit = X_y.drop('image_path', axis=1)
X_y_emb_vit.head(50)

X = X_y_emb_vit.drop('y', axis=1)
y = X_y_emb_vit['y']

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"Y_train shape: {Y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Y_test shape: {Y_test.shape}")


   no_of_comments           t  follower_count_at_t                image_path  \
0               0  1594174009             40934474  ../Data/insta_data/0.jpg   
1               0  1593571666             40934474  ../Data/insta_data/2.jpg   
2               0  1593136341             40934474  ../Data/insta_data/4.jpg   
3               0  1592981047             40934474  ../Data/insta_data/6.jpg   
4               0  1592703461             40934474  ../Data/insta_data/8.jpg   

   y                                      vit_embedding  
0  1  [-1.3519323, 0.7332789, 1.3956515, 0.99564266,...  
1  1  [0.9231997, -0.61542, -1.0848525, -0.4418159, ...  
2  1  [0.8908843, -1.5599669, -1.7691066, -0.5515488...  
3  1  [1.4084183, -0.30265957, 0.7955862, 0.02323306...  
4  1  [1.0409024, -0.7152434, -0.4358073, -1.2306467...  
X_train shape: (3023, 4)
Y_train shape: (3023,)
X_test shape: (756, 4)
Y_test shape: (756,)


In [ ]:
numerical_features = ['no_of_comments', 't', 'follower_count_at_t']

scaler = StandardScaler()

X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test[numerical_features] = scaler.transform(X_test[numerical_features])

print(X_train.head())
print(X_test.head())



      no_of_comments         t  follower_count_at_t  \
3552       -0.090845 -1.409514            -0.978271   
543        -0.108466  0.642249             1.003118   
2365       -0.103556  0.445742            -0.522049   
3319        0.042372  0.387155            -0.737797   
1615       -0.051334 -0.802690             0.038099   

                                          vit_embedding  
3552  [0.02374128, 0.750116, -0.8317369, 1.8580775, ...  
543   [-0.4026929, 0.048733443, 0.9171584, 0.1790615...  
2365  [0.38576534, 0.74692, -0.9205371, -0.07063876,...  
3319  [-0.26024455, -0.7170677, -0.33554626, 0.56067...  
1615  [-0.0149905, 0.15502143, -0.32345766, 0.997068...  
      no_of_comments         t  follower_count_at_t  \
1292        0.381860 -2.125342            -0.017016   
2620       -0.104611 -0.022089            -0.696068   
2361       -0.092267  0.669443            -0.522049   
298        -0.109200 -0.226284             1.517566   
1126       -0.110761  0.703900             0.2

In [ ]:
numerical_inputs = Input(shape=(len(numerical_features),))
vit_embedding_input = Input(shape=(768,))

numerical_dense = Dense(128, activation='relu')(numerical_inputs)

merged_layer = concatenate([numerical_dense, vit_embedding_input])

dense1 = Dense(64, activation='relu')(merged_layer)
dropout_layer = tf.keras.layers.Dropout(0.5)(dense1) # Add a dropout layer to prevent overfitting
output_layer = Dense(3, activation='softmax')(dropout_layer)

model = Model(inputs=[numerical_inputs, vit_embedding_input], outputs=output_layer)
model.summary()

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 3)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │            512 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 768)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 896)            │              0 │ dense[0][0],           │
│                           │                        │                │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 64)             │         57,408 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 64)             │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 3)              │            195 │ dropout[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 58,115 (227.01 KB)

 Trainable params: 58,115 (227.01 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

X_train_numerical = X_train[numerical_features]
X_train_vit_embeddings = np.array(X_train['vit_embedding'].tolist())
X_test_numerical = X_test[numerical_features]
X_test_vit_embeddings = np.array(X_test['vit_embedding'].tolist())


# Train the model
model.fit([X_train_numerical, X_train_vit_embeddings], Y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate([X_test_numerical, X_test_vit_embeddings], Y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

Epoch 1/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.3723 - loss: 1.3966 - val_accuracy: 0.4347 - val_loss: 1.0529
Epoch 2/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4936 - loss: 1.0041 - val_accuracy: 0.4612 - val_loss: 1.0289
Epoch 3/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5919 - loss: 0.8792 - val_accuracy: 0.4826 - val_loss: 0.9900
Epoch 4/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6435 - loss: 0.7810 - val_accuracy: 0.4694 - val_loss: 0.9971
Epoch 5/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6922 - loss: 0.7162 - val_accuracy: 0.4777 - val_loss: 0.9882
Epoch 6/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7344 - loss: 0.6382 - val_accuracy: 0.5207 - val_loss: 0.9811
Epoch 7/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7596 - loss: 0.5837 - val_accuracy: 0.5157 - val_loss: 0.9869
Epoch 8/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8163 - loss: 0.4977 - val_accuracy: 0.5140 - v